In [ ]:
# !pip install stanza
# !pip install ollama
# !pip install vertexai
# !pip install -t lib google-auth google-auth-httplib2 google-api-python-client --upgrade
# !pip install pandas_gbq
# !pip install python-dotenv

In [1]:
import pandas as pd
import os
import re
import stanza
import random
import logging
import pymorphy3
import json
import shutil

from tqdm import tqdm

tqdm.pandas()

import numpy as np
import spacy
# import tqdm
# from langchain_text_splitters import RecursiveCharacterTextSplitter

nlp = spacy.load("uk_ner_web_trf_13class")

/Users/linndfors/study/diploma/new_venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/linndfors/study/diploma/new_venv/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/linndfors/study/diploma/new_venv/lib/python3.10/site-packages/spacy_transformers/layers/hf_shim.py:137: UserWarning: Error loading saved torch state_dict with strict=True, likely due to differences between 'transformers' versions. Attempting to load with strict=False as a fallback...

If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current 'transformers' and 'spacy-transforme

## LLM

In [2]:
import openai
from openai import OpenAI

import os
from dotenv import load_dotenv

load_dotenv(".env")

openai.api_key = os.environ.get("OPENAI_API_KEY")
first_names_with_sex_df = pd.read_csv("first_names_frequency_popular.csv")

In [3]:
def fill_prompt_with_names(female_names, male_names):
    updated_prompt = f"""
Ви - експерт з обробки природної мови. Ваше завдання - змінити задане речення зі зміною статі, використовуючи словник анотацій у форматі {{"слово": "його сутність(label) з NER"}}. Дотримуйтесь таких правил:
    1. Якщо зустрічається слово чоловічого роду з класу "JOB", змініть його на жіночий аналог, і навпаки для жіночого роду. 
       **Виняток**: якщо ця сутність класу "JOB" належить до **спільного роду** (наприклад, "голова", "суддя"), такі слова, а також усі пов'язані з ними залежні слова (імена, прикметники, дієслова тощо), **залишаються без змін**.
    2. Якщо "JOB" було змінене, то змінити потрібно і сутність "PERS".
    3. Якщо слова не відносятся до сутностей в анотаціях, то їх змінювати непотрібно.
    Знайди в сутності "PERS" імʼя і:
    а) якщо імʼя належить до чоловічого роду то зміни на жіноче, обране лише з наступних запропонованих варіантів: {female_names}.
    б) якщо імʼя належить до жіночого роду то зміни на чоловіче, обране лише з наступних запропонованих варіантів: {male_names}.
    Не можна використосувати інші імена, окрім запропонованих.
    4. Зберігайте відмінки слів: якщо слово було, наприклад, у родовому відмінку ("режисера"), то його аналог має теж бути в родовому відмінку ("режисерки").
    5. Зберігайте форму слова за числом: множина залишається множиною ("власники" → "власниці"), однина - одниною.
    6. Забезпечте граматичну коректність речення, узгоджуючи змінені слова з контекстом.
    7. Залиши всі інші знаки, пунктуації, символи, включаючи символи "<" і ">", незмінними.
    8. Як результат виведи змінене речення без жодних додаткових пояснень.

    Приклад:
        Input: Дмитро Петренко став новим адміністратором за наказом заступниці директора Тетяни Сірої, яка належить до прихильників радикальних методів. {{"Дмитро Петренко": "PERS", "Тетяни Сірої": "PERS", "адміністратором": "JOB", "заступниці": "JOB", "директора": "JOB"}}
        Output: Марина Петренко стала новою адміністраторкою за наказом заступника директорки Володимира Сірого, який належить до прихильників радикальних методів.

        Input: Сестри художника Бориса Мартинюка були відомими співачками, тому у Бориса не було часу проводити час з подругами. {{"художника": "JOB", "Бориса Мартинюка": "PERS", "співачками": "JOB", "Бориса": "PERS"}}
        Output: Брати художниці Олени Мартинюк були відомими співаками, тому у Олени не було часу проводити час з подругами.
        
        Input: Суддя Марина зʼявилась запізно <.{{"Суддя": "JOB", "Марина": "PERS"}}
        Otput: Суддя Марина зʼявилась запізно <.
"""
    return updated_prompt

In [4]:
def generate_random_name(df, n, sex=None):
    if sex:
        df = df[df['sex'] == sex]
    
    names = df['name']
    frequencies = df['freq']
    
    return list(np.random.choice(names, size=n, p=frequencies/np.sum(frequencies)))


def change_gender(sentence):
    male_names = generate_random_name(first_names_with_sex_df, 3, 'M')
    female_names = generate_random_name(first_names_with_sex_df, 3, 'F')
    print("male names:", male_names)
    print("female names:", female_names)

    gender_swap_prompt = fill_prompt_with_names(female_names, male_names)

    messages = [
        {"role": "system", "content": gender_swap_prompt},
        {"role": "user", "content": sentence},
    ]

    response = openai.chat.completions.create(
        model="gpt-4o-mini", messages=messages, temperature=0.8, max_tokens=250, top_p=1
    )

    response_content = response.choices[0].message.content
    return response_content

# Prepare data

In [5]:
def find_ann_files(directory):
    ann_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".ann"):
                file_path = os.path.join(root, file)
                ann_files.append(file_path)

    return ann_files

def read_ann_files(directory):
    ann_files = find_ann_files(directory)
    job_title_sentences = []

    for ann_file in ann_files:
        with open(ann_file, "r", encoding="utf-8") as ann_f:
            for ann_l in ann_f:
                ann_row = ann_l.split("\t")

                try:
                    if ann_row[1] == "JOB":
                        job_title_sentences.append(ann_row)
                except Exception as e:
                    continue

    return job_title_sentences

def find_txt_files(directory):
    txt_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                txt_files.append(file_path)

    return txt_files

def read_txt_files(directory):
    txt_files = find_txt_files(directory)
    text = []

    for txt_file in txt_files:
        with open(txt_file, "r", encoding="utf-8") as txt_f:
            text.append(txt_f.read())

    return text

text_ng = read_txt_files("/Users/linndfors/study/diploma/ner_for_fem/data/ng")
text_bruk = read_txt_files("/Users/linndfors/study/diploma/ner_for_fem/data/bruk")

print("texts bruk:", len(text_bruk))
print("chars bruk:", len("".join(text_bruk)))

texts bruk: 0
chars bruk: 0


In [6]:
def parse_ann_file(ann_file):
    entities = []

    with open(ann_file, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) < 3:
                continue
            label = parts[1]
            start = parts[2]
            end = parts[3]
            text = parts[4]

            entities.append({"text": text, "start": start, "end": end, "label": label})

    return entities


def extract_entities(txt_file, ann_file):
    labeled_sentences_list = []
    sentences = []

    with open(txt_file, "r", encoding="utf-8") as f:
        sentence_lines = f.readlines()

    entities = parse_ann_file(ann_file)

    for sentence in sentence_lines:

        labeled_dict = {}
        job = False
        sentence = sentence.strip()

        for ent in entities:
            if re.search(rf"\b{re.escape(ent['text'])}\b", sentence):
                labeled_dict[ent["text"]] = ent["label"]
                if ent["label"] == "JOB":
                    job = True

        if job:
            labeled_sentences_list.append(labeled_dict)
            sentences.append(sentence)

    return labeled_sentences_list, sentences

# def get_sentence_with_job_titles(ann_files):

#     sentence_with_jobs = {}

#     for ann_file in ann_files:
#         txt_file = ann_file.replace(".ann", ".txt")
#         labeled_sentences_list, sentences = extract_entities(txt_file, ann_file)
#         for i in range(len(sentences)):
#             sentence_with_jobs[sentences[i]] = labeled_sentences_list[i]

#     return sentence_with_jobs

# ng_ann_files = find_ann_files("/Users/linndfors/study/diploma/ner_for_fem/data/ng")
# ng_sentence_with_jobs = get_sentence_with_job_titles(ng_ann_files)

# bruk_ann_files = find_ann_files("/Users/linndfors/study/diploma/ner_for_fem/data/bruk")
# bruk_sentence_with_jobs = get_sentence_with_job_titles(bruk_ann_files)

In [7]:
def read_data(file_path):
    data_dict = {}
    current_label = None

    with open(file_path, "r") as file:
        for line in file:
            line = line.strip()

            if line in ["DEV", "TEST"]:
                current_label = line
                data_dict[current_label] = []
            elif current_label:
                if line:
                    data_dict[current_label].append(line)

    return data_dict

file_path = "/Users/linndfors/study/diploma/ner_for_fem/data/dev-test-split.txt"
data_dict = read_data(file_path)
# print(data_dict)

In [8]:
def find_files(directory):
    dev_list = []
    test_list = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                if file.split(".")[0] in list(data_dict["DEV"]):
                    file_path = os.path.join(root, file)
                    dev_list.append(file_path)
                elif file.split(".")[0] in list(data_dict["TEST"]):
                    file_path = os.path.join(root, file)
                    test_list.append(file_path)

    return dev_list, test_list


def read_files(txt_files):
    text = ""
    for txt_file in txt_files:
        with open(txt_file, "r", encoding="utf-8") as txt_f:
            text += txt_f.read()

    return text

In [66]:
bruk_dev, bruk_test = find_files("/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk")
ng_dev, ng_test = find_files("/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng")

dev_files = bruk_dev + ng_dev
test_files = bruk_test + ng_test

ng_files = ng_dev + ng_test
bruk_files = bruk_dev + bruk_test 

dev_files_text = read_files(dev_files)
test_files_text = read_files(test_files)

dev_ng_text = read_files(ng_dev)
test_ng_text = read_files(ng_test)

dev_bruk_text = read_files(bruk_dev)
test_bruk_text = read_files(bruk_test)

In [67]:
def get_sentences_with_job_titles(txt_file_names):

    sentence_with_jobs = {}

    for txt_file in txt_file_names:
        ann_file = txt_file.replace(".txt", ".ann")
        labeled_sentences_list, sentences = extract_entities(txt_file, ann_file)
        for i in range(len(sentences)):
            sentence_with_jobs[sentences[i]] = labeled_sentences_list[i]

    return sentence_with_jobs

test_sentences_with_jobs = get_sentences_with_job_titles(test_files)
dev_sentences_with_jobs = get_sentences_with_job_titles(dev_files)

In [68]:
# with open("/Users/linndfors/study/diploma/ner_for_fem/JSON_files/test_sentences_with_jobs.json", "w", encoding="utf-8") as f:
#     json.dump(test_sentences_with_jobs, f, ensure_ascii=False, indent=4)

# with open("/Users/linndfors/study/diploma/ner_for_fem/JSON_files/dev_sentences_with_jobs.json", "w", encoding="utf-8") as f:
#     json.dump(dev_sentences_with_jobs, f, ensure_ascii=False, indent=4)

# Run Gender-swapping

In [ ]:
# gender_swaped_pairs_dev = {}

# for sentence, ann_dict in tqdm(
#     dev_sentences_with_jobs.items(), desc="Processing sentences"
# ):
#     try:
#         filtered_ann_dict = {key: value for key, value in ann_dict.items() if value in {'PERS', 'JOB'}}
#         dev_raw_and_annotated = sentence + "\n" + str(filtered_ann_dict)
#         # print(test_raw_and_annotated)
#         output = change_gender(dev_raw_and_annotated)
#         print(dev_raw_and_annotated)
#         print(output)
#         print("-------------------------------------")
#         gender_swaped_pairs_dev[sentence] = output
#     except Exception as e:
#         print("error:", e)

Processing sentences:   0%|          | 0/1024 [00:00<?, ?it/s]

male names: ['Володимир', 'Руслан', 'Денис']
female names: ['Ніна', 'Валентина', 'Вікторія']


Processing sentences:   0%|          | 1/1024 [00:02<44:37,  2.62s/it]

Виставу за п'єсою російського класика Льва Толстого поставили у столичному Театрі імені Франка . 28 квітня тут відбулася прем'єра спектаклю « Живий труп » у постановці режисера Романа Мархоліа .
{'Льва Толстого': 'PERS', 'режисера': 'JOB', 'Романа Мархоліа': 'PERS', 'Толстого': 'PERS', 'Мархоліа': 'PERS'}
Виставу за п'єсою російського класика Льва Толстої поставили у столичному Театрі імені Франка . 28 квітня тут відбулася прем'єра спектаклю « Живий труп » у постановці режисерки Вікторії Мархоліа .
-------------------------------------
male names: ['Олексій', 'Андрій', 'Юрій']
female names: ['Галина', 'Юлія', 'Ганна']


Processing sentences:   0%|          | 2/1024 [00:04<36:10,  2.12s/it]

...

За останні чотири місяці суд зобов'язав вже щонайменше сім фірм повернути Генічеській райдержадміністрації пляжні зони, оскільки по суті бізнесмени отримали безкоштовно землю біля моря, на якій заробляли.
{'бізнесмени': 'JOB'}
За останні чотири місяці суд зобов'язав вже щонайменше сім фірм повернути Генічеській райдержадміністрації пляжні зони, оскільки по суті бізнесменки отримали безкоштовно землю біля моря, на якій заробляли.
-------------------------------------
male names: ['Леонід', 'Василь', 'Михайло']
female names: ['Марина', 'Ганна', 'Наталія']


Processing sentences: 100%|█████████▉| 1023/1024 [35:10<00:01,  1.97s/it]

Засновниками ТОВ «Енергогазінвестконсалт» є Олена Смик з Глевахи і столичне ТОВ «Обрій», засновницею якого є та ж Смик.
{'Засновниками': 'JOB', 'Олена Смик': 'PERS', 'засновницею': 'JOB', 'Смик': 'PERS'}
Засновницями ТОВ «Енергогазінвестконсалт» є Леонід Смик з Глевахи і столичне ТОВ «Обрій», засновником якого є та ж Смик.
-------------------------------------
male names: ['Ігор', 'Олег', 'Євген']
female names: ['Людмила', 'Пелагея', 'Вікторія']


Processing sentences: 100%|██████████| 1024/1024 [35:10<00:00,  2.06s/it]

Директором фірми є Смик Сергій Миколайович.
{'Смик': 'PERS', 'Директором': 'JOB', 'Смик Сергій Миколайович': 'PERS'}
Директоркою фірми є Смик Сергій Миколайович.
-------------------------------------


In [79]:
gender_swaped_df = pd.DataFrame(
    gender_swaped_pairs_dev.items(), 
    columns=["original", "swapped"]
)

In [80]:
gender_swaped_df.head()

,original,swapped
0,Виставу за п'єсою російського класика Льва Тол...,Виставу за п'єсою російського класика Льва Тол...
1,Віртуозні Лесь Задніпровський – князь Абрезков...,Віртуозні Ганна Задніпровська – княгиня Абрезк...
2,"Старша в Києві , одружена з військовим , чолов...","Старша в Києві , одружена з військовою , чолов..."
3,"Ні , не так - автор статті , позаштатний корес...","Ні , не так - авторка статті , позаштатна коре..."
4,І лектори в сірих піджаках із червоними парткв...,І лектори в сірих піджаках із червоними парткв...


In [81]:
df_name = "/Users/linndfors/study/diploma/ner_for_fem/FOR_ANNOTATION_FINAL/dev_swapped_sentences_with_names_frequencies.csv"
gender_swaped_df.to_csv(df_name)

Add old annotation

In [ ]:
# # read swapped sentences

# orig_and_changed_dict = {}

# with open("/Users/linndfors/study/diploma/ner_for_fem/FOR_ANNOTATION_FINAL/dev_swapped_sentences_with_names_frequencies.csv", mode='r', encoding='utf-8') as file:
#     reader = csv.reader(file)
    
#     next(reader)
#     for row in reader:
#         key = row[1]
#         value = row[2]
#         orig_and_changed_dict[key] = value

# print(orig_and_changed_dict)

# with open("/Users/linndfors/study/diploma/ner_for_fem/JSON_files/test_sentences_with_jobs.json", "r", encoding="utf-8") as f:
#     test_sentences_with_jobs = json.load(f)

# with open("/Users/linndfors/study/diploma/ner_for_fem/JSON_files/dev_sentences_with_jobs.json", "r", encoding="utf-8") as f:
#     dev_sentences_with_jobs = json.load(f)

# def set_annotattions(original_sent):
#     if original_sent in dev_sentences_with_jobs.keys():
#         annotations = dev_sentences_with_jobs[original_sent] 
#         filtered_annotations = {key: value for key, value in annotations.items() if value in {'PERS', 'JOB'}}
#         return filtered_annotations
#     return None

# swapped_df['old_Annotations'] = swapped_df.apply(lambda x: set_annotattions(x['original']), axis=1)
# swapped_df.to_csv("/Users/linndfors/study/diploma/ner_for_fem/FOR_ANNOTATION_FINAL/dev_swapped_with_ann_with_names_changed.csv", index=False)

{"Виставу за п'єсою російського класика Льва Толстого поставили у столичному Театрі імені Франка . 28 квітня тут відбулася прем'єра спектаклю « Живий труп » у постановці режисера Романа Мархоліа .": "Виставу за п'єсою російського класика Льва Толстої поставили у столичному Театрі імені Франка . 28 квітня тут відбулася прем'єра спектаклю « Живий труп » у постановці режисерки Вікторії Мархоліа .", 'Віртуозні Лесь Задніпровський – князь Абрезков та Олег Стальчук – Слідчий , Геній .': 'Віртуозні Ганна Задніпровська – княгиня Абрезкова та Юрій Стальчук – Слідчий, Геній.', 'Старша в Києві , одружена з військовим , чоловік нещодавно дістав підполковника , і донька дуже з цього тішиться .': 'Старша в Києві , одружена з військовою , чоловік нещодавно дістав підполковника , і донька дуже з цього тішиться .', 'Ні , не так - автор статті , позаштатний кореспондент « Ровесника » , худий патлатий хлопчина в окулярах із грубими скельцями , просто побачив у неї на столі кілька мухоморів , нічого не за

# Annotations reseting

In [51]:
dev_swapped_df = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/src/dev_swapped.csv")
test_swapped_df = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/src/test_swapped.csv")

swapped_df = pd.concat([dev_swapped_df, test_swapped_df], ignore_index=True)

In [52]:
swapped_df.head()

,Оригінальне речення,Змінене речення,Анотації,Коректність речення,Виправлене речення,Помічник/ця
0,Виставу за п'єсою російського класика Льва Тол...,Виставу за п'єсою російського класика Льва Тол...,"{'Льва Толстого': 'PERS', 'режисера': 'JOB', '...",Містить помилки,Виставу за п'єсою російської класикині Марії Т...,Оля Н
1,Віртуозні Лесь Задніпровський – князь Абрезков...,Віртуозні Ганна Задніпровська – княгиня Абрезк...,"{'Лесь Задніпровський': 'PERS', 'князь': 'JOB'...",Містить помилки,Віртуозні Ганна Задніпровська – княгиня Абрезк...,NaN
2,"Старша в Києві , одружена з військовим , чолов...","Старша в Києві , одружена з військовою , чолов...",{'військовим': 'JOB'},Містить помилки,"Старша в Києві , одружена з військовою , жінка...",NaN
3,"Ні , не так - автор статті , позаштатний корес...","Ні , не так - авторка статті , позаштатна коре...",{'позаштатний кореспондент': 'JOB'},Правильне,NaN,NaN
4,І лектори в сірих піджаках із червоними парткв...,І лектори в сірих піджаках із червоними парткв...,{'священиків': 'JOB'},Правильне,NaN,NaN


In [53]:
swapped_df['gender-swapped'] = swapped_df.apply(lambda x: x['Виправлене речення'] if x['Виправлене речення'] and x['Коректність речення']=='Містить помилки' else x['Змінене речення'], axis=1)

In [54]:
swapped_df[(swapped_df['Коректність речення']=='Містить помилки') & (swapped_df['Виправлене речення'].isna())]

,Оригінальне речення,Змінене речення,Анотації,Коректність речення,Виправлене речення,Помічник/ця,gender-swapped


In [55]:
swapped_df.head()

,Оригінальне речення,Змінене речення,Анотації,Коректність речення,Виправлене речення,Помічник/ця,gender-swapped
0,Виставу за п'єсою російського класика Льва Тол...,Виставу за п'єсою російського класика Льва Тол...,"{'Льва Толстого': 'PERS', 'режисера': 'JOB', '...",Містить помилки,Виставу за п'єсою російської класикині Марії Т...,Оля Н,Виставу за п'єсою російської класикині Марії Т...
1,Віртуозні Лесь Задніпровський – князь Абрезков...,Віртуозні Ганна Задніпровська – княгиня Абрезк...,"{'Лесь Задніпровський': 'PERS', 'князь': 'JOB'...",Містить помилки,Віртуозні Ганна Задніпровська – княгиня Абрезк...,NaN,Віртуозні Ганна Задніпровська – княгиня Абрезк...
2,"Старша в Києві , одружена з військовим , чолов...","Старша в Києві , одружена з військовою , чолов...",{'військовим': 'JOB'},Містить помилки,"Старша в Києві , одружена з військовою , жінка...",NaN,"Старша в Києві , одружена з військовою , жінка..."
3,"Ні , не так - автор статті , позаштатний корес...","Ні , не так - авторка статті , позаштатна коре...",{'позаштатний кореспондент': 'JOB'},Правильне,NaN,NaN,"Ні , не так - авторка статті , позаштатна коре..."
4,І лектори в сірих піджаках із червоними парткв...,І лектори в сірих піджаках із червоними парткв...,{'священиків': 'JOB'},Правильне,NaN,NaN,І лектори в сірих піджаках із червоними парткв...


In [56]:
orig_and_changed_dict = dict(zip(swapped_df['Оригінальне речення'], swapped_df['gender-swapped']))

In [57]:
# change original text with the sentences we gender-swapped (sentences with JOB TITLEs)
all_files_text = dev_files_text + test_files_text
all_sent = all_files_text.split("\n")


print("Sentence to change:", len(all_sent))

Sentence to change: 24047


In [249]:
def format_punctuation(text):
    # Step 1: Remove extra spaces before punctuation marks (if any)
    text = re.sub(r'\s+([,()?!:])', r'\1', text)
    text = re.sub(r'\s+(\.)', r'\1', text)
    
    # Step 2: Add space before punctuation marks if not already present
    text = re.sub(r'(?<!\s)([,.()?!:])', r' \1', text)
    
    # Step 3: Add space after punctuation marks if not already present (except dot)
    text = re.sub(r'([,()?!:])(?!\s)', r'\1 ', text)
    
    # Step 4: Remove double spaces created by previous steps
    text = re.sub(r'\s{2,}', ' ', text)
    
    return text

In [360]:
def change_orig_text_to_gend_swapped(txt_file, list_of_manually_changed_files):
    # create a new files with the old name but new text with swapped sentences. also add old ann file
    file_and_text_dict = {}
    file_need_to_be_changed = None

    ann_file = txt_file.replace(".txt", ".ann")
    changed_ann_file = ann_file.replace('ng', "ng_changed")
    changed_ann_file= changed_ann_file.replace('bruk', "bruk_changed")
    shutil.copyfile(ann_file, changed_ann_file)

    with open(txt_file, "r", encoding="utf-8") as txt_f:
        text = txt_f.read()

        new_text = []

        for x in text.split("\n"):
            if x:
                if x in orig_and_changed_dict.keys():
                    # corrected_text = re.sub(r'(?<=\w)([,.:!?;])', r' \1', orig_and_changed_dict[x])
                    new_text.append(orig_and_changed_dict[x])
                    file_need_to_be_changed = txt_file
                else:
                    new_text.append(x)
            else:
                new_text.append("")

    
    changed_txt_file = txt_file.replace('ng', "ng_changed")
    changed_txt_file = changed_txt_file.replace('bruk', "bruk_changed")
    if new_text:
        # print(new_text)
        if txt_file not in list_of_manually_changed_files:
            joined_text = "\n".join(new_text)
            file_and_text_dict[changed_txt_file] = joined_text
        else:
            with open(changed_txt_file, 'r') as file:
                changed_text = file.read()
            file_and_text_dict[changed_txt_file] = changed_text    
            print("add:", changed_txt_file)    

    return file_and_text_dict, file_need_to_be_changed

In [350]:
def dict_from_ann_files(directory):
    ann_files = find_ann_files(directory)
    ann_files_content = {}

    for ann_file in ann_files:
        with open(ann_file, "r", encoding="utf-8") as ann_f:
            rows = []
            buffer = []
            for line in ann_f:
                line = line.strip()
                if "\t" in line:
                    if buffer:
                        rows.append(parse_annotation_line("\n".join(buffer)))
                        buffer = []
                    buffer.append(line)
                else:
                    buffer.append(line)
            
            if buffer:
                rows.append(parse_annotation_line("\n".join(buffer)))

        ann_files_content[ann_file] = rows

    return ann_files_content

def parse_annotation_line(line):
    """
    Parses a single annotation line, including multi-line entities.
    """
    try:
        parts = line.split("\t", maxsplit=4)
        ind, label, start, end = parts[:4]
        ent = parts[4] if len(parts) > 4 else ""
        return {"ind": ind, "label": label, "start": int(start), "end": int(end), "ent": ent.replace("\n", " ")}
    except Exception as e:
        print(f"Error parsing annotation line: {line}")
        return None


In [390]:
from difflib import SequenceMatcher

def tokenize_with_indices(text):
    """Tokenizes text and returns a list of (token, start, end)."""
    doc = nlp(text)
    return [(token.text, token.idx, token.idx + len(token)) for token in doc]

def create_new_ann_file(orig_file_name, changed_file_name, changed_text, ann_files):
    with open(orig_file_name, 'r') as file:
        original_text = file.read()

    old_ann_file = orig_file_name.replace(".txt", ".ann")
    original_annotations = ann_files[old_ann_file]

    original_tokens = tokenize_with_indices(original_text)
    changed_tokens = tokenize_with_indices(changed_text)

    dict_for_changed_ann = {}
    index_offset_map = {}
    current_offset = 0
    orig_char_index = 0
    changed_char_index = 0

    orig_token_index = 0
    changed_token_index = 0

    while orig_char_index < len(original_text) and changed_char_index < len(changed_text):
        index_offset_map[orig_char_index] = current_offset

        if orig_token_index < len(original_tokens) and changed_token_index < len(changed_tokens):
            orig_token = original_tokens[orig_token_index]
            changed_token = changed_tokens[changed_token_index]

            if orig_char_index == orig_token[1] and changed_char_index == changed_token[1]:
                if orig_token[0] != changed_token[0]:
                    print("offest changed for: ", changed_token[0], orig_token[0])
                    current_offset += len(changed_token[0]) - len(orig_token[0])

                orig_char_index += len(orig_token[0])
                changed_char_index += len(changed_token[0])

                if orig_char_index == orig_token[2]:
                    orig_token_index += 1
                if changed_char_index == changed_token[2]:
                    changed_token_index += 1
            else:
                orig_char_index += 1
                changed_char_index += 1
        else:
            index_offset_map[orig_char_index] = current_offset
            orig_char_index += 1

    while orig_char_index < len(original_text):
        index_offset_map[orig_char_index] = current_offset
        orig_char_index += 1

    for annotation in original_annotations:
        orig_start = annotation['start']
        orig_end = annotation['end']
        new_start = orig_start + index_offset_map.get(orig_start, 0)
        new_end = orig_end + index_offset_map.get(orig_end, 0)

        new_ent = changed_text[new_start:new_end]
        dict_for_changed_ann[annotation['ind']] = (new_ent, (new_start, new_end))

    return dict_for_changed_ann

In [61]:
# # ORIGINAL 

# def create_new_ann_file(orig_file_name, changed_file_name, changed_text, ann_files):
#     with open (orig_file_name, 'r') as file:
#         original_text = file.read()

#     old_ann_file = orig_file_name.replace(".txt", ".ann")
#     original_annotations = ann_files[old_ann_file]

#     original_tokens = tokenize_with_indices(original_text)
#     changed_tokens = tokenize_with_indices(changed_text)

#     dict_for_changed_ann = {}
#     offset = 0
#     for idx, (orig_token, changed_token) in enumerate(zip(original_tokens, changed_tokens)):
#         if orig_token != changed_token:
#             # print(f"{orig_token} ---- {changed_token}")
#             # print(changed_token[1], orig_token[1])
            
#             # if orig_token[1] == 1605:
#             #     print("-------HERE-------")
#             #     print(orig_token, "-----------", changed_token)        
#             for annotation in original_annotations:
#                 if annotation['start'] + offset == changed_token[1]:
#                     # print("found relevant entity with the same start:", annotation['ent'])
#                     # знайшли ентіті з анотацій
#                     if changed_token[2] != annotation['end'] + offset:
#                         # це ентіті змінилось
#                         # print("this token changed its end index")
#                         if changed_token[2] == (annotation['end'] + (len(changed_token[0]) - len(orig_token[0]))):
#                             # просто змінилось саме слово
#                             # print("---edit old ent (1 ent = 1 token):", changed_token[0], (changed_token[1], changed_token[2]))
#                             new_ent = changed_token[0]
#                             dict_for_changed_ann[annotation['ind']] = (new_ent, (changed_token[1], changed_token[2]))
#                         else:
#                             # print ("---this token is a part of more than 2 tokens entity")
#                             counter = 0
#                             difference = 0
#                             while idx + counter + 1 < len(changed_tokens) and idx + counter +1 < len(original_tokens):
#                                 # print(len(original_tokens[idx + counter][0]))
#                                 # print("changed tokens:", changed_tokens)
#                                 # print("len:", len(original_tokens))
#                                 # print("original tokens:", original_tokens)
#                                 difference += (len(changed_tokens[idx + counter][0]) - len(original_tokens[idx + counter][0]))
#                                 # print("difference:", difference)
#                                 # print("current end:", changed_tokens[idx + counter][2])
#                                 if changed_tokens[idx + counter][2] == annotation['end'] + offset + difference:
#                                     ind_change = idx + counter + 1
#                                     new_ent_list = changed_tokens[idx:ind_change]
#                                     new_ent = " ".join(ent[0] for ent in new_ent_list)
#                                     new_ind = (changed_tokens[idx][1], changed_tokens[idx + counter][2])
#                                     dict_for_changed_ann[annotation['ind']] = (new_ent, new_ind)
#                                     # print("new ent:", (new_ent, new_ind))
#                                     break
#                                 counter += 1
#                     else:
#                         # це ентіті не змінилось, а лише посунулось
#                         # print("offset:", offset)
#                         # print("update old ent (the same ent, but moved by offset):", changed_token[0], (annotation['start'] + offset, annotation['end'] + offset))
#                         dict_for_changed_ann[annotation['ind']] = (changed_token[0], (annotation['start'] + offset, annotation['end'] + offset))
#             offset += len(changed_token[0]) - len(orig_token[0])
#             # print("offset:", offset)



#     return dict_for_changed_ann

In [273]:
ann_files = dict_from_ann_files("/Users/linndfors/study/diploma/ner_for_fem/data/ng")

In [283]:
# # ann_files = dict_from_ann_files("/Users/linndfors/study/diploma/ner_for_fem/data/ng")
# # copy_ann_files = ann_files
# new_ann_dict = {}

# files_need_to_be_changed = set()

# for old_file in tqdm.tqdm(bruk_files[132:]):
#     print(old_file)
#     ann_file = old_file.replace(".txt", ".ann")
#     # ann_file = ann_file.replace("ng", "ng_changed")
#     new_file_new_dev_dict, file_need_to_be_changed = change_orig_text_to_gend_swapped(old_file)

#     if file_need_to_be_changed:
#         files_need_to_be_changed.add(file_need_to_be_changed)

#     for file_path, content in new_file_new_dev_dict.items():
#         os.makedirs(os.path.dirname(file_path), exist_ok=True)
        
#         with open(file_path, 'w', encoding='utf-8') as file:
#             file.write(content)

#         dict_for_changed_ann = create_new_ann_file(old_file, file_path, content, ann_files)

#         for x in ann_files[ann_file]:
#             if x['ind'] in dict_for_changed_ann.keys():
#                 x['end'] = dict_for_changed_ann[x['ind']][1][1]
#                 x['start'] = dict_for_changed_ann[x['ind']][1][0]
#                 new_ent = dict_for_changed_ann[x['ind']][0]
#                 x['ent'] = new_ent


In [ ]:
def write_ann_files_from_dict(ann_files_content):
    for ann_file, annotations in tqdm.tqdm(ann_files_content.items()):
        relevant_txt_file = ann_file.replace(".ann", ".txt")
        if relevant_txt_file in files_need_to_be_changed:
            ann_file = ann_file.replace("bruk/", "bruk_changed/")
            with open(ann_file, "w", encoding="utf-8") as f:
                for annotation in annotations:
                    line = f"{annotation['ind']}\t{annotation['label']}\t{annotation['start']}\t{annotation['end']}\t{annotation['ent']}\n"
                    f.write(line)

write_ann_files_from_dict(ann_files)

In [ ]:
dir1 = "/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner_new/ng_changed"
dir2 = "/Users/linndfors/study/diploma/ner_for_fem/data/ng"

count1 = len([f for f in os.listdir(dir1) if os.path.isfile(os.path.join(dir1, f))])
count2 = len([f for f in os.listdir(dir2) if os.path.isfile(os.path.join(dir2, f))])

if count1 == count2:
    print("Both directories have the same number of files:", count1)
else:
    print(f"Different file counts - {dir1}: {count1}, {dir2}: {count2}")

Different file counts - /Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner_new/ng_changed: 524, /Users/linndfors/study/diploma/ner_for_fem/data/ng: 596


# Testing

In [ ]:
# list_of_manually_changed_files_bruk = [
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/2298b6b41bec.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/e21ddb9efb07.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/6b5d4229b548.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/4b1efc986bb8.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/9df8bcf8e17e.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/e023d6d6f1a3.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/f958ce8bb461.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/b87576a7425a.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/2846bf1d66a8.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/c0c436cc9cb6.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/e1d8841dcc68.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/ff069c11e13d.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/f4136bfda19a.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/f5bb37569413.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/ef1b6bd52747.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/e010e86e436c.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/d530671576f8.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/cee02d4c030f.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/b019e9b8d51d.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/326978d2c58c.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/52049bc57554.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/548f66f38090.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/547ce681ada8.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/87ae82667e2a.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/71a651d2dbec.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/27a1722d517c.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/9df8bcf8e17e.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/8cc49a8b2d76.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/7b2c9399fee9.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/6b60e6fb3d78.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/2fc890744f7c.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/1c325fde9ed4.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/0d902430768c.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/0b962bee49bc.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/0ac5140eb732.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/647a3f078f3f.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/38603af07530.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/dc1202fd9850.txt',
#  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/e5e76a8efa0f.txt' 
#  ]

In [371]:
list_of_manually_changed_files_ng =[
    '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/c97bff36f45a.txt',
    '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/39c7437146fd.txt',
    '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/c99d1ad0cc3c.txt',
    '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/d509463360e2.txt',
    '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/1e8b049462e1.txt',
    '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/00b1a798bebf.txt',
    '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/cdc9c08df7ac.txt',
    '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/4f2aecdde4c8.txt',
    '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/5e33850771e3.txt',
    '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/f5e3415a3653.txt',
    '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/3aa7c6a81ddd.txt',
    '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/b645c9653d72.txt',
    '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/82e415f0dd48.txt',
    '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/0f94c5cc63c3.txt',
    '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/f2653a1e387d.txt',
    '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/a04af354c8db.txt',
]

In [162]:
# key_file = '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/e1d8841dcc68.txt'

# for x in range(len(files_with_broken_tokens_count[10][key_file][0])):
#     a = files_with_broken_tokens_count[10][key_file][0][x]
#     b = files_with_broken_tokens_count[10][key_file][1][x]
    
#     if a[0] != b[0]:
#         print(a, b)

In [ ]:
# ann_files = dict_from_ann_files("/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng")
files_to_change = ng_files

In [ ]:
# list_to_check = ['/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/2fc890744f7c.txt',
#                  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/0ac5140eb732.txt',
#                  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/0b962bee49bc.txt',
#                  '/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/bruk/0d902430768c.txt']

In [392]:
ann_files_1 = dict_from_ann_files("/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng")
# ner_for_fem/data/orig_data/bruk_changed/e1d8841dcc68.ann

In [393]:
new_ann_dict = {}

files_need_to_be_changed = set()

for old_file in tqdm(['/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/5e33850771e3.txt']):
    print(old_file)
    ann_file = old_file.replace(".txt", ".ann")
    # ann_file = ann_file.replace("ng", "ng_changed")
    # назва файлу і змінений текст, назва файла, який був змінений або None, якщо не змінювався
    new_file_new_dev_dict,          file_need_to_be_changed = change_orig_text_to_gend_swapped(old_file, list_of_manually_changed_files_ng)

    if file_need_to_be_changed:
        files_need_to_be_changed.add(file_need_to_be_changed)

    for file_path, content in new_file_new_dev_dict.items():
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(content)

        dict_for_changed_ann = {}
        if old_file in files_need_to_be_changed:
            dict_for_changed_ann = create_new_ann_file(old_file, file_path, content, ann_files_1)

        for x in ann_files_1[ann_file]:
            if x['ind'] in dict_for_changed_ann.keys():
                x['end'] = dict_for_changed_ann[x['ind']][1][1]
                x['start'] = dict_for_changed_ann[x['ind']][1][0]
                new_ent = dict_for_changed_ann[x['ind']][0]
                x['ent'] = new_ent


  0%|          | 0/1 [00:00<?, ?it/s]

/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/5e33850771e3.txt
add: /Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng_changed/5e33850771e3.txt


100%|██████████| 1/1 [00:08<00:00,  8.50s/it]

offest changed for:  Заступниця Заступник
offest changed for:  Марина Дмитро
offest changed for:  задекларувала задекларував
offest changed for:  прокурорка прокурор
offest changed for:  Марина Дмитро
offest changed for:  починала починав
offest changed for:  прокуроркою прокурором
offest changed for:  входила входив
offest changed for:  адвокатки адвоката
offest changed for:  Олени Сергія
offest changed for:  Власенко Власенка
offest changed for:  яка який
offest changed for:  судилася судився
offest changed for:  своїм своєю
offest changed for:  колишнім колишньою
offest changed for:  чоловіком дружиною
offest changed for:  Василем Наталією
offest changed for:  Окунським Окунською
offest changed for:  прокурорка прокурор
offest changed for:  зазначала зазначав
offest changed for:  її його
offest changed for:  останній останньої
offest changed for:  прокурорки прокурора
offest changed for:  прокурорки прокурора
offest changed for:  Олени Борзих
offest changed for:  нею ним
offest chan

In [394]:
ann_files_1['/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/5e33850771e3.ann']

[{'ind': 'T1',
  'label': 'JOB',
  'start': 0,
  'end': 42,
  'ent': 'Заступниця Головного військового прокурора'},
 {'ind': 'T2',
  'label': 'ORG',
  'start': 43,
  'end': 74,
  'ent': 'Генеральної прокуратури України'},
 {'ind': 'T3',
  'label': 'PERS',
  'start': 75,
  'end': 88,
  'ent': 'Марина Борзих'},
 {'ind': 'T4',
  'label': 'DATE',
  'start': 110,
  'end': 123,
  'ent': 'минулого року'},
 {'ind': 'T5', 'label': 'LOC', 'start': 147, 'end': 156, 'ent': 'Луганську'},
 {'ind': 'T6', 'label': 'JOB', 'start': 187, 'end': 197, 'ent': 'прокурорка'},
 {'ind': 'T7', 'label': 'LOC', 'start': 244, 'end': 250, 'ent': 'Києвом'},
 {'ind': 'T8', 'label': 'ART', 'start': 265, 'end': 271, 'ent': 'Prosud'},
 {'ind': 'T9',
  'label': 'PERS',
  'start': 274,
  'end': 287,
  'ent': 'Марина Борзих'},
 {'ind': 'T10',
  'label': 'JOB',
  'start': 310,
  'end': 321,
  'ent': 'прокуроркою'},
 {'ind': 'T11',
  'label': 'LOC',
  'start': 322,
  'end': 353,
  'ent': 'Ленінського району м. Луганська'},
 {

In [367]:
def write_ann_files_from_dict(ann_files_content):
    for ann_file, annotations in tqdm(ann_files_content.items()):
        relevant_txt_file = ann_file.replace(".ann", ".txt")
        if relevant_txt_file in files_need_to_be_changed:
            # ann_file = ann_file.replace("bruk/", "bruk_changed/")
            ann_file = ann_file.replace("ng/", "ng_changed/")
            with open(ann_file, "w", encoding="utf-8") as f:
                for annotation in annotations:
                    line = f"{annotation['ind']}\t{annotation['label']}\t{annotation['start']}\t{annotation['end']}\t{annotation['ent']}\n"
                    f.write(line)

write_ann_files_from_dict(ann_files)

100%|██████████| 298/298 [00:00<00:00, 1221.33it/s]


In [387]:
with open('/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng/a04af354c8db.txt', 'r') as file:
        original_text = file.read()

with open('/Users/linndfors/study/diploma/ner_for_fem/data/orig_data/ng_changed/a04af354c8db.txt', 'r') as file:
        changed_text = file.read()

In [388]:
original_tokens = tokenize_with_indices(original_text)
changed_tokens = tokenize_with_indices(changed_text)

In [389]:
for x in range(len(original_tokens)):
    if original_tokens[x][0] != changed_tokens[x][0]:
        print(original_tokens[x], "---", changed_tokens[x])